# Natural language to SQL

**Run in [Google Colab](https://colab.research.google.com/) For GPU.**

This model have  Mistral as a base and it has been fine-tuned to excel in SQL code generation.

In [2]:
from huggingface_hub import login
login(userdata.get("HF_TOKEN"))


In [3]:
#Install the lastest versions of peft & transformers library recommended
#if you want to work with the most recent models
!pip install -q git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.0 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/accelerate.git to /tmp/p

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import accelerate

In [5]:
model_name = "defog/sqlcoder-7b"

We need to create the Quantization configuration to load the Model.

It is a large model and I want it to fit in a 16GB GPU, I'm going to use a 4 bits quantization.

If you want to learn more about quantization, refer to this article: [QLoRA: Training a Large Language Model on a 16GB GPU.](https://medium.com/towards-artificial-intelligence/qlora-training-a-large-language-model-on-a-16gb-gpu-00ea965667c1)

You can try to use this model in a 8 bit quantizations and check in you see any improvements in the results.

In [6]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)


To load the model I pass to the AutoModelForCasualLM teh quantization configurations, and HuggingFace take care of all the hard work.

In [7]:
foundation_model = AutoModelForCausalLM.from_pretrained(model_name,
                    quantization_config=bnb_config,
                    device_map='auto',
                    use_cache = True)

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

self.pre_quantized False


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]

tokenizer_config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

This function wraps the call to *model.generate*

In [9]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=400):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        num_return_sequences=1,
        eos_token_id=eos_token_id,
        pad_token_id=eos_token_id,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=5
    )
    return outputs

# Prompt without Shots.
In this first PROMPT we are going to give Instructions to the model and pass the structure of the Database.

The instructions are significantly different from those we are passing to GPT-3.5-Turbo. This model is really well fine-tuned, but it is smaller than GPT-3.5.

We need to be more clear with the instructions, as it does not have the same capacity to understand our orders as GPT-3.5.

In [28]:
sp_nl2sql = """
    ### Instructions:
Your task is to convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question.

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    TABLES HERE:

-- Table: students
CREATE TABLE students (
    student_id INT PRIMARY KEY,  -- Unique ID for each student
    name TEXT,                   -- Full name of the student
    gender TEXT,                 -- Gender of the student
    birth_date DATE,             -- Date of birth
    enrollment_year INT          -- The year student enrolled
);

INSERT INTO students VALUES
(1, 'Aisha', 'F', '2002-07-14', 2020),
(2, 'Omar', 'M', '2001-11-30', 2019),
(3, 'Laila', 'F', '2003-03-21', 2021),
(4, 'Fahad', 'M', '2000-01-08', 2018);

-- Table: courses
CREATE TABLE courses (
    course_id INT PRIMARY KEY,  -- Unique ID for each course
    course_name TEXT,           -- Name of the course
    credits INT                 -- Number of credit hours
);

INSERT INTO courses VALUES
(101, 'Database Systems', 3),
(102, 'Linear Algebra', 4),
(103, 'Artificial Intelligence', 3),
(104, 'English Composition', 2);

-- Table: grades
CREATE TABLE grades (
    student_id INT,             -- Student taking the course
    course_id INT,              -- Course taken by the student
    semester TEXT,              -- Semester when course was taken
    grade TEXT,                 -- Grade received
    FOREIGN KEY (student_id) REFERENCES students(student_id),
    FOREIGN KEY (course_id) REFERENCES courses(course_id)
);

INSERT INTO grades VALUES
(1, 101, 'Fall 2021', 'A'),
(1, 102, 'Spring 2022', 'B'),
(2, 103, 'Fall 2021', 'A'),
(3, 101, 'Fall 2022', 'C'),
(4, 104, 'Spring 2022', 'B');

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
"""


In [29]:
sp_nl2sql = sp_nl2sql.format(question="List the names of students who received a grade 'A' in any course.")
print(sp_nl2sql)



    ### Instructions:
Your task is to convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question.

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    TABLES HERE:

-- Table: students
CREATE TABLE students (
    student_id INT PRIMARY KEY,  -- Unique ID for each student
    name TEXT,                   -- Full name of the student
    gender TEXT,                 -- Gender of the student
    birth_date DATE,             -- Date of birth
    enrollment_year INT          -- The year student enrolled
);

INSERT INTO students VALUES
(1, 'Aisha', 'F', '2002-07-14', 2020),
(2, 'Omar', 'M', '2001-11-30', 2019),
(3, 'Laila', 'F', '2003-03-21', 2021),
(4, 'Fahad', 'M', '2000-01-08', 2018);

-- Table: courses
CREATE TABLE courses (
    course_id INT 

In [30]:
input_sentences = tokenizer(sp_nl2sql, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)

In [31]:
#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

In [32]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT students.name FROM students JOIN grades ON students.student_id = grades.student_id JOIN courses ON grades.course_id = courses.course_id WHERE grades.grade = 'A' ORDER BY students.name NULLS LAST;


The SQL Order is correct.

#Prompt with shots OpenAI Style.
In this second prompt we are going to add some Shots with samples to see if our SQL style affects the model.

In [33]:
sp_nl2sql2 = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about the Databases structure**

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    -- Table: employees
    CREATE TABLE employees (
        ID_usr INT PRIMARY KEY,
        name TEXT
    );

    -- Table: salary
    CREATE TABLE salary (
        ID_usr INT,
        year INT,
        base FLOAT,
        bonus FLOAT,
        FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
    );

    INSERT INTO employees VALUES
    (1, 'Alice'),
    (2, 'Bob'),
    (3, 'Charlie');

    INSERT INTO salary VALUES
    (1, 2022, 50000, 1000),
    (2, 2022, 70000, 3000),
    (3, 2022, 60000, 1500);

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `Return the name of the best paid employee`:
    ```sql3
    SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_usr = s.ID_usr
    ORDER BY s.base DESC
    LIMIT 1;
    ```
"""

print(sp_nl2sql2)



    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about the Databases structure**

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    -- Table: employees
    CREATE TABLE employees (
        ID_usr INT PRIMARY KEY,  
        name TEXT
    );

    -- Table: salary
    CREATE TABLE salary (
        ID_usr INT,              
        year INT,                
        base FLOAT,              
        bonus FLOAT,             
        FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
    );

    INSERT INTO employees VALUES
    (1, 'Alice'),
    (2, 'Bob'),
    (3, 'Charlie');

    INSERT INTO salary VALUES
    (1, 2022, 50000, 1000),


In [34]:
sp_nl2sql2 = sp_nl2sql2.format(question="Return The name of the best paid employee")
(print(sp_nl2sql2))


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about the Databases structure**

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    -- Table: employees
    CREATE TABLE employees (
        ID_usr INT PRIMARY KEY,  
        name TEXT
    );

    -- Table: salary
    CREATE TABLE salary (
        ID_usr INT,              
        year INT,                
        base FLOAT,              
        bonus FLOAT,             
        FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
    );

    INSERT INTO employees VALUES
    (1, 'Alice'),
    (2, 'Bob'),
    (3, 'Charlie');

    INSERT INTO salary VALUES
    (1, 2022, 50000, 1000),


In [35]:
input_sentences = tokenizer(sp_nl2sql2, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [18]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT employees.first_name, employees.last_name, MAX(employees.salary) AS max_salary FROM employees GROUP BY employees.first_name, employees.last_name ORDER BY max_salary DESC NULLS LAST LIMIT 1;


The Order is really different from the one obtained with the first prompt.

The first difference is the format. But The SQL is realy more simple, at least it is my sensation.

#Prompt with Shots in Sample Style.

In this prompt, we will place the examples in a separate section, and in the instructions, we will instruct the model to pay attention to them in order to generate the SQL commands.

In [36]:
sp_nl2sql3b = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    -- Table: employees
    CREATE TABLE employees (
        ID_usr INT PRIMARY KEY,
        name TEXT
    );

    -- Table: salary
    CREATE TABLE salary (
        ID_usr INT,
        year INT,
        base FLOAT,
        bonus FLOAT,
        FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
    );

    INSERT INTO employees VALUES
    (1, 'Alice'),
    (2, 'Bob'),
    (3, 'Charlie');

    INSERT INTO salary VALUES
    (1, 2022, 50000, 1000),
    (2, 2022, 70000, 3000),
    (3, 2022, 60000, 1500);

    ### Samples

    -- Question: Return the name of all employees
    SELECT name FROM employees;

    -- Question: Return the employee names and their base salaries
    SELECT e.name, s.base
    FROM employees e
    JOIN salary s ON e.ID_usr = s.ID_usr;

    -- Question: Return the average salary of employees
    SELECT AVG(base) FROM salary;

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `Return the name of the best paid employee`:
    ```sql3
    SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_usr = s.ID_usr
    ORDER BY s.base DESC
    LIMIT 1;
    ```
"""

print(sp_nl2sql3b)



    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    -- Table: employees
    CREATE TABLE employees (
        ID_usr INT PRIMARY KEY,  
        name TEXT
    );

    -- Table: salary
    CREATE TABLE salary (
        ID_usr INT,              
        year INT,                
        base FLOAT,              
        bonus FLOAT,             
        FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
    );

    INSERT INTO employees VALUES
    (1, 'Alice'),
    (2, 'Bob'),
    (3, 'Charlie');

    INSERT INTO salary VALUES
    (1, 2022, 50000, 1000),
   

In [37]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Return The name of the best paid employee")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    -- Table: employees
    CREATE TABLE employees (
        ID_usr INT PRIMARY KEY,  
        name TEXT
    );

    -- Table: salary
    CREATE TABLE salary (
        ID_usr INT,              
        year INT,                
        base FLOAT,              
        bonus FLOAT,             
        FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
    );

    INSERT INTO employees VALUES
    (1, 'Alice'),
    (2, 'Bob'),
    (3, 'Charlie');

    INSERT INTO salary VALUES
    (1, 2022, 50000, 1000),
   

In [38]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [39]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_usr = s.ID_usr
    ORDER BY s.base DESC
    LIMIT 1;


#Now the question in Arabic.


In [40]:
sp_nl2sql3 = sp_nl2sql3b.format(question="أرجِع اسم الموظف الأعلى أجرًا")
print(sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    -- Table: employees
    CREATE TABLE employees (
        ID_usr INT PRIMARY KEY,  
        name TEXT
    );

    -- Table: salary
    CREATE TABLE salary (
        ID_usr INT,              
        year INT,                
        base FLOAT,              
        bonus FLOAT,             
        FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
    );

    INSERT INTO employees VALUES
    (1, 'Alice'),
    (2, 'Bob'),
    (3, 'Charlie');

    INSERT INTO salary VALUES
    (1, 2022, 50000, 1000),
   

In [41]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [42]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_usr = s.ID_usr
    ORDER BY s.base DESC
    LIMIT 1;


The generated SQL command is the same regardless of where we have placed the examples.

#Conclusions.

Let's see the three SQL's together.

* SELECT employees.name, MAX(salary.salary) AS max_salary FROM employees JOIN salary ON employees.ID_Usr = salary.ID_Usr GROUP BY employees.name ORDER BY max_salary DESC NULLS LAST LIMIT 1;

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* Spanish Question: SELECT e.name
     FROM employees e
     JOIN salary s ON e.ID_Usr = s.ID_Usr
     WHERE s.salary = (SELECT MAX(salary) FROM salary)
     GROUP BY e.name
     ORDER BY COUNT(studies.ID_study) DESC
     LIMIT 1;


**The model has demonstrated that it is highly efficient in crafting SQL.** Additionally, it pays a lot of attention, perhaps too much, to the examples we provide. Clearly, these examples should be crafted by one of the best SQL programmers we have access to, though their use may not be essential.

On the other hand, although the model is clearly very proficient in SQL generation, during the creation of the notebook, I have encountered several issues because the commands need to be extremely clear. It doesn't handle typos well (which should not exist).

It appears to have some issues when it receives commands in Spanish. I assume this problem would be present in any language other than English. Therefore, since it's a tool that could be used by non-technical personnel, this should be considered in environments where English is not the primary language.

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [44]:
sp_nl2sql3b = """
    ### Instructions:
Your task is to convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL in the ### Samples section to learn more about the database structure**

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    -- Table: customers
    CREATE TABLE customers (
        customer_id INT PRIMARY KEY,
        name TEXT,
        email TEXT,
        join_date DATE
    );

    -- Table: products
    CREATE TABLE products (
        product_id INT PRIMARY KEY,
        product_name TEXT,
        category TEXT,
        price FLOAT
    );

    -- Table: orders
    CREATE TABLE orders (
        order_id INT PRIMARY KEY,
        customer_id INT,
        order_date DATE,
        total_amount FLOAT,
        FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
    );

    -- Table: order_items
    CREATE TABLE order_items (
        order_item_id INT PRIMARY KEY,
        order_id INT,
        product_id INT,
        quantity INT,
        FOREIGN KEY (order_id) REFERENCES orders(order_id),
        FOREIGN KEY (product_id) REFERENCES products(product_id)
    );

    ### Samples

    -- Q: Return the names of all customers who bought products in the 'Electronics' category
    SELECT DISTINCT c.name
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
    JOIN products p ON oi.product_id = p.product_id
    WHERE p.category = 'Electronics';

    -- Q: Return the total amount spent by each customer
    SELECT c.name, SUM(o.total_amount) AS total_spent
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    GROUP BY c.name;

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `Return the names of customers who bought products in the 'Electronics' category`:
    ```sql3
    SELECT DISTINCT c.name
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
    JOIN products p ON oi.product_id = p.product_id
    WHERE p.category = 'Electronics';
    ```
"""

print(sp_nl2sql3b)



    ### Instructions:
Your task is to convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL in the ### Samples section to learn more about the database structure**

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    -- Table: customers
    CREATE TABLE customers (
        customer_id INT PRIMARY KEY,
        name TEXT,
        email TEXT,
        join_date DATE
    );

    -- Table: products
    CREATE TABLE products (
        product_id INT PRIMARY KEY,
        product_name TEXT,
        category TEXT,
        price FLOAT
    );

    -- Table: orders
    CREATE TABLE orders (
        order_id INT PRIMARY KEY,
        customer_id INT,
        order_date DATE,
        total_amount FLOAT,
        FOREIGN KEY (cu

In [45]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Return the names of customers who bought products in the 'Electronics' category")
print(sp_nl2sql3)



    ### Instructions:
Your task is to convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL in the ### Samples section to learn more about the database structure**

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    -- Table: customers
    CREATE TABLE customers (
        customer_id INT PRIMARY KEY,
        name TEXT,
        email TEXT,
        join_date DATE
    );

    -- Table: products
    CREATE TABLE products (
        product_id INT PRIMARY KEY,
        product_name TEXT,
        category TEXT,
        price FLOAT
    );

    -- Table: orders
    CREATE TABLE orders (
        order_id INT PRIMARY KEY,
        customer_id INT,
        order_date DATE,
        total_amount FLOAT,
        FOREIGN KEY (cu

In [46]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()


print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT c.name, SUM(o.total_amount) AS total_spent
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    GROUP BY c.name;


In [47]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Return the names of customers who purchased products in the 'Clothing' category")
print(sp_nl2sql3)



    ### Instructions:
Your task is to convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL in the ### Samples section to learn more about the database structure**

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    -- Table: customers
    CREATE TABLE customers (
        customer_id INT PRIMARY KEY,
        name TEXT,
        email TEXT,
        join_date DATE
    );

    -- Table: products
    CREATE TABLE products (
        product_id INT PRIMARY KEY,
        product_name TEXT,
        category TEXT,
        price FLOAT
    );

    -- Table: orders
    CREATE TABLE orders (
        order_id INT PRIMARY KEY,
        customer_id INT,
        order_date DATE,
        total_amount FLOAT,
        FOREIGN KEY (cu

In [48]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()


print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT c.name, SUM(o.total_amount) AS total_spent
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    GROUP BY c.name;


In [49]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Return the names of customers who ordered products that cost more than $500")
print(sp_nl2sql3)



    ### Instructions:
Your task is to convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL in the ### Samples section to learn more about the database structure**

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    -- Table: customers
    CREATE TABLE customers (
        customer_id INT PRIMARY KEY,
        name TEXT,
        email TEXT,
        join_date DATE
    );

    -- Table: products
    CREATE TABLE products (
        product_id INT PRIMARY KEY,
        product_name TEXT,
        category TEXT,
        price FLOAT
    );

    -- Table: orders
    CREATE TABLE orders (
        order_id INT PRIMARY KEY,
        customer_id INT,
        order_date DATE,
        total_amount FLOAT,
        FOREIGN KEY (cu

In [50]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()


print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT c.name, SUM(o.total_amount) AS total_spent
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    GROUP BY c.name;
